# Look at accelerometer data 

Finding Zero velocity times by rail axis acceleration noise levels, making summary statistics for the noise levels across the whole day files.  Spot check graphs to see what works

In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [2]:
#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform

from time import time as ti

In [3]:
import CoreFunctions as cf
from skimage.restoration import denoise_wavelet

## Choosing Platform
Working is beinging conducted on several computers, and author needs to be able to run code on all without rewriting..  This segment of determines which computer is being used, and sets the directories accordingly.

In [4]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

In [5]:
if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'

In [6]:
if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/Recordings2/'
    imageFolder = '/sciclone/scr10/dchendrickson01/Move3Dprint/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'

## Global Variables

In [7]:
Saving = False
location = folder
Titles = True
Ledgends = True

f = 0


In [8]:
files = ['230418 recording1.csv','230419 recording1.csv','230420 recording1.csv','230421 recording1.csv',
         '230418 recording2.csv','230419 recording2.csv','230420 recording2.csv','230421 recording2.csv']

In [9]:
BeforeTamping = ['221206 recording1.csv','221207 recording1.csv','221208 recording1.csv','221209 recording1.csv',
         '221206 recording2.csv','221207 recording2.csv','221208 recording2.csv','221209 recording2.csv']


## Project Specific Functions

In [10]:
def RollingStdDev(RawData, SmoothData, RollSize = 25):
    StdDevs = []
    for i in range(RollSize):
        Diffs = RawData[0:i+1]-SmoothData[0:i+1]
        Sqs = Diffs * Diffs
        Var = sum(Sqs) / (i+1)
        StdDev = np.sqrt(Var)
        StdDevs.append(StdDev)
    for i in range(len(RawData)-RollSize-1):
        j = i + RollSize
        Diffs = RawData[i:j]-SmoothData[i:j]
        Sqs = Diffs * Diffs
        Var = sum(Sqs) / RollSize
        StdDev = np.sqrt(Var)
        StdDevs.append(StdDev)  
    
    return StdDevs

def RollingSum(Data, Length = 100):
    RollSumStdDev = []
    for i in range(Length):
        RollSumStdDev.append(sum(Data[0:i+1]))
    for i in range(len(Data) - Length):
        RollSumStdDev.append(sum(Data[i:i+Length]))
    return RollSumStdDev

def SquelchPattern(DataSet, StallRange = 5000, SquelchLevel = 0.02):
    SquelchSignal = np.ones(len(DataSet))

    for i in range(len(DataSet)-2*StallRange):
        if np.average(DataSet[i:i+StallRange]) < SquelchLevel:
            SquelchSignal[i+StallRange]=0

    return SquelchSignal

def getVelocity(Acceleration, Timestamps = 0.003, Squelch = [], corrected = 0):
    velocity = np.zeros(len(Acceleration))
    
    Acceleration -= np.average(Acceleration)
    
    if len(Timestamps) == 1:
        dTime = np.ones(len(Acceleration),dtype=float) * Timestamps
    elif len(Timestamps) == len(Acceleration):
        dTime = np.zeros(len(Timestamps), dtype=float)
        dTime[0]=1
        for i in range(len(Timestamps)-1):
            j = i+1
            if Timestamps[j] > Timestamps[i]:
                dTime[j]=Timestamps[j]-Timestamps[i]
            else:
                dTime[j]=Timestamps[j]-Timestamps[i]+10000.0
        dTime /= 10000.0

    velocity[0] = Acceleration[0] * (dTime[0])

    for i in range(len(Acceleration)-1):
        j = i + 1
        if corrected ==2:
            if Squelch[j]==0:
                velocity[j]=0
            else:
                velocity[j] = velocity[i] + Acceleration[j] * dTime[j]                
        else:
            velocity[j] = velocity[i] + Acceleration[j] * dTime[j]

    if corrected == 1:
        PointVairance = velocity[-1:] / len(velocity)
        for i in range(len(velocity)):
            velocity[i] -=  PointVairance * i
    
    velocity *= 9.81

    return velocity

def MakeDTs(Seconds, Miliseconds):
    dts = np.zeros(len(Miliseconds), dtype=float)
    dts[0]=1
    for i in range(len(MiliSeconds)-1):
        j = i+1
        if Seconds[j]==Seconds[i]:
            dts[j]=Miliseconds[j]-Miliseconds[i]
        else:
            dts[j]=Miliseconds[j]-Miliseconds[i]+1000
    dts /= 10000
    return dts



In [11]:
#Smooth = cf.Smoothing(ODataSet[:,3],2) #,50)
def DeviationVelocity(file):
    if file[-3:] =='csv':
        ODataSet = np.genfromtxt(open(folder+file,'r'), delimiter=',',skip_header=0,missing_values=0,invalid_raise=False)
        SmoothX = denoise_wavelet(ODataSet[:,3], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothY = denoise_wavelet(ODataSet[:,4], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothZ = denoise_wavelet(ODataSet[:,5], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothX -= np.average(SmoothX)
        SmoothY -= np.average(SmoothY)
        SmoothZ -= np.average(SmoothZ)
        StdDevsX = RollingStdDev(ODataSet[:,3],SmoothX)
        StdDevsX.append(0)
        StdDevsX = np.asarray(StdDevsX)
        SmoothDevX = denoise_wavelet(StdDevsX, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SquelchSignal = SquelchPattern(SmoothDevX, 2000, 0.03)
        #Velocity = getVelocity(ODataSet[:,3], ODataSet[:,2],SquelchSignal, 2)
        #Velocity = np.asarray(Velocity)
        MoveMatrix = np.matrix([SmoothX, SmoothY, SmoothZ])
        return [SquelchSignal,MoveMatrix,SmoothDevX,file[:-4]]
    else:
        pass


In [12]:
#Smooth = cf.Smoothing(ODataSet[:,3],2) #,50)
def SmoothMoves(file):
    if file[-3:] =='csv':
        ODataSet = np.genfromtxt(open(folder+file,'r'), delimiter=',',skip_header=0,missing_values=0,invalid_raise=False)
        SmoothX = denoise_wavelet(ODataSet[:,3], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothY = denoise_wavelet(ODataSet[:,4], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothZ = denoise_wavelet(ODataSet[:,5], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothX -= np.average(SmoothX)
        SmoothY -= np.average(SmoothY)
        SmoothZ -= np.average(SmoothZ)
        MoveMatrix = np.matrix([SmoothX, SmoothY, SmoothZ])
        return MoveMatrix
    else:
        pass


In [13]:
def SepreateMovements(SquelchSignal, RawData, FileName):
    Moves= []
    MoveNames = []
    Move = np.zeros((1,3), dtype=float)
    i = 0
    for j in range(len(SquelchSignal)-1):
        if SquelchSignal[j] == 1:
            #try:
            Move = np.concatenate((Move, RawData[j,:]), axis=0)
            #except:
            #    print(j)
            if SquelchSignal[j+1] == 0:
                #Move = np.matrix(Move)
                Moves.append(Move)
                MoveNames.append(FileName + str(i).zfill(3))
                i+=1
                Move = np.zeros((1,3), dtype=float)
                #Move[0,2]=0
    Moves.append(Move)
    MoveNames.append(FileName + str(i).zfill(3))
    return Moves, MoveNames
    


In [14]:
def splitLong(Moves, maxLength = 4000, minLength = 1000, MoveNames = []):
    if len(MoveNames) <=1:
        MoveNames = ['null'  for x in range(len(Moves))]
    Xmoves = []
    Xnames = []
    for i in range(len(Moves)):
        if np.shape(move)[0] > maxLength: 
            Xmoves.append(Moves[i][:int(len(Moves[i])/2),:])
            Xnames.append(MoveNames[i] + 'a')
            Xmoves.append(Moves[i][int(len(Moves[i])/2):,:])
            Xnames.append(MoveNames[i] + 'b')
        else:
            if np.shape(Moves[i])[0] < minLength:
                pass
            else:
                Xmoves.append(Moves[i])
                Xnames.append(MoveNames[i])
    return Xmoves, Xnames

def findMaxLength(Moves):
    maxLength = 0
    LongMove = 0
    for i in range(len(Moves)):
        if np.shape(Moves[i])[0] > maxLength: 
            maxLength =  np.shape(Moves[i])[0]
            LongMove = i
    return maxLength, LongMove

def findMinLength(Moves):
    minLength = 9999999
    SmallMove = 0
    for i in range(len(Moves)):
        if np.shape(Moves[i])[0] < minLength: 
            minLength =  np.shape(Moves[i])[0]
            SmallMove = i
    return minLength, SmallMove



## Process Files

In [15]:
LoopFiles = 8
loops = int(len(files) / LoopFiles) 
if len(files)%LoopFiles != 0:
    loops += 1


In [16]:
SquelchSignal = []
RawData=[]
OrderedFileNames=[]


In [30]:
loops

1

In [31]:
st = ti()

Moves = []

for k in range(loops):
    if k == loops -1:
        tfiles = files[k*LoopFiles:]
    else:
        tfiles = files[k*LoopFiles:(k+1)*LoopFiles]
    #Results = Parallel(n_jobs=LoopFiles)(delayed(DeviationVelocity)(file) for file in tfiles)
    Results = Parallel(n_jobs=LoopFiles)(delayed(SmoothMoves)(file) for file in tfiles)
    #Results =[]
    #for file in tfiles:
    #    Results.append(DeviationVelocity(file))
    #    print(file, (ti()-st)/60.0)
        
    #for i in range(len(Results)):       
    #    SquelchSignal.append(Results[i][0])
    #    RawData.append(Results[i][1])
    #    OrderedFileNames.append(Results[i][3])
    #print(k, np.shape(Results), (ti()-st)/60.0)
    for result in Results:
        Moves.append(result)
    print(k, (ti()-st)/60.0)    



0 1.9833999594052634


In [20]:
#MoveData = Parallel(n_jobs=31)(delayed(SepreateMovements)(SquelchSignal[i], RawData[i], OrderedFileNames[i])
#                                       for i in range(len(RawData)))

MoveData = []
for i in range(len(RawData)):
    MoveData.append(SepreateMovements(SquelchSignal[i], RawData[i].T, OrderedFileNames[i]))

In [21]:
Movements = []
GroupNames = []
for move in MoveData:
    Movements.append(move[0])
    GroupNames.append(move[1])



In [22]:
Moves=[]
for Groups in Movements:
    for Move in Groups:
        Moves.append(np.asarray(Move).astype('float32'))
#Moves = np.asarray(Moves)

MoveNames = []
for Groups in GroupNames:
    for name in Groups:
        MoveNames.append(name)

In [23]:

del SquelchSignal
del RawData
del Movements
del GroupNames
del MoveData
del OrderedFileNames


In [24]:
longMove, MoveNumb = findMaxLength(Moves)


In [25]:
minLength = 750

In [26]:
Moves, MoveNames = splitLong(Moves, longMove+1, minLength, MoveNames)


In [33]:
np.shape(Moves[0])

(3, 18659125)

In [ ]:
#padding moves.  Not needed, need sequences, not moves

#Moves2 = []
#for move in Moves:
#    if np.shape(move)[0] < longMove:
#        padding = np.zeros((longMove-np.shape(move)[0], 3))
#        tempMove = np.concatenate((move, padding), axis=0)
#        Moves2.append(tempMove)
#    else:
#        Moves2.append(move)
#Moves = Moves2

#del Moves2

## Try LSTM Stuff

https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [86]:
TimeSteps = 250
StepSize = 50
Features = np.shape(Moves[0])[0]

In [87]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps, s_step = 1):
    X, y = list(), list()
    Steps_to_take = int(len(sequences) / s_step)
    for j in range(Steps_to_take):
        i = j * s_step
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [112]:
Sequences = []
Outputs = []
for move in Moves:
    Seq, Out = split_sequences(move.T,TimeSteps,StepSize)
    Sequences.append(Seq)
    Outputs.append(Out)
    

In [113]:
MoveSegments = []
for seq in Sequences:
    for mv in seq:
        MoveSegments.append(mv)
NextDataPoint = []
for out in Outputs:
    for pt in out:
        NextDataPoint.append(np.reshape(pt,(1,3)))

In [114]:
len(MoveSegments)

2570895

In [115]:
from sklearn.model_selection import train_test_split

In [116]:
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Masking
from keras.models import Sequential
import tensorflow as tf


class LSTM_Autoencoder:
  def __init__(self, optimizer='adam', loss='mse'):
    self.optimizer = optimizer
    self.loss = loss
    self.n_features = Features
    self.timesteps = TimeSteps
    
  def build_model(self):
    timesteps = self.timesteps
    n_features = self.n_features
    model = Sequential()
    
    # Padding
    #model.add(Masking(mask_value=0.0, input_shape=(timesteps, n_features)))

    # Encoder
    model.add(LSTM(timesteps, activation='relu', input_shape=(TimeSteps, Features), return_sequences=True))
    model.add(LSTM(35, activation='relu', return_sequences=True))
    model.add(LSTM(6, activation='relu'))
    model.add(RepeatVector(timesteps))
    
    # Decoder
    model.add(LSTM(timesteps, activation='relu', return_sequences=True))
    model.add(LSTM(35, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
    model.summary()
    self.model = model
    
  def fit(self, X, epochs=3, batch_size=32):
    #self.timesteps = np.shape(X)[0]
    self.build_model()
    
    #input_X = np.expand_dims(X, axis=1)
    self.model.fit(X, X, epochs=epochs, batch_size=batch_size)
    
  def predict(self, X):
    #input_X = np.expand_dims(X, axis=1)
    output_X = self.model.predict(input_X)
    reconstruction = np.squeeze(output_X)
    return np.linalg.norm(X - reconstruction, axis=-1)
  
  def plot(self, scores, timeseries, threshold=0.95):
    sorted_scores = sorted(scores)
    threshold_score = sorted_scores[round(len(scores) * threshold)]
    
    plt.title("Reconstruction Error")
    plt.plot(scores)
    plt.plot([threshold_score]*len(scores), c='r')
    plt.show()
    
    anomalous = np.where(scores > threshold_score)
    normal = np.where(scores <= threshold_score)
    
    plt.title("Anomalies")
    plt.scatter(normal, timeseries[normal][:,-1], s=3)
    plt.scatter(anomalous, timeseries[anomalous][:,-1], s=5, c='r')
    plt.show()


In [117]:
#split = int(len(Moves)*.9)
#Train_data = Moves[:split]
#Test_data = Moves[split:]
#Train_data = tf.ragged.constant(Train_data)
#Test_data = tf.ragged.constant(Test_data)

In [118]:
lstm_autoencoder2 = LSTM_Autoencoder(optimizer='adam', loss='msle')

In [119]:
lstm_autoencoder2.build_model()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_26 (LSTM)              (None, 250, 250)          254000    
                                                                 
 lstm_27 (LSTM)              (None, 250, 35)           40040     
                                                                 
 lstm_28 (LSTM)              (None, 6)                 1008      
                                                                 
 repeat_vector_5 (RepeatVect  (None, 250, 6)           0         
 or)                                                             
                                                                 
 lstm_29 (LSTM)              (None, 250, 250)          257000    
                                                                 
 lstm_30 (LSTM)              (None, 250, 35)           40040     
                                                      

In [120]:
Batches = 32
NumbBatches = 1000

SamplesPerSet = Batches * NumbBatches


In [121]:
SetsNeeded = int(len(MoveSegments) / SamplesPerSet)
print(len(MoveSegments), SetsNeeded)

2570895 80


In [122]:
PercentPerSet = 1.0 / float(SetsNeeded)
if len(Moves) % SamplesPerSet != 0:
    SetsNeeded += 1

PercentHoldOutForNext=1.0

In [123]:
st = ti()

for i in range(SetsNeeded-1):
    PercentHoldOutForNext = 1.0 - PercentPerSet / PercentHoldOutForNext
    seq_train, seq_test, out_train, out_test = train_test_split(MoveSegments, NextDataPoint, test_size=PercentHoldOutForNext, shuffle=True, random_state=0)
    seq_train = np.asarray(seq_train)
    lstm_autoencoder2.model.fit(seq_train, seq_train, epochs=2, batch_size=32, verbose=1)
    MoveSegments = seq_test
    NextDataPoint = out_test
    print(str(i+1)+' of ' + str(SetsNeeded), (ti()-st)/60/60, (((ti()-st)/(i+1) * ( SetsNeeded -1) - (ti()-st) )/60/60))
    

Epoch 1/2
1005/1005 [==============================] - 855s 845ms/step - loss: nan - accuracy: 0.3852
Epoch 2/2
1005/1005 [==============================] - 853s 849ms/step - loss: nan - accuracy: 0.3409
1 of 81 28.56851975520452 37.61521777682834
Epoch 1/2
1005/1005 [==============================] - 858s 854ms/step - loss: nan - accuracy: 0.3410
Epoch 2/2
1005/1005 [==============================] - 853s 849ms/step - loss: nan - accuracy: 0.3410
2 of 81 57.13698284626007 37.139038952986404
Epoch 1/2
992/992 [==============================] - 834s 841ms/step - loss: nan - accuracy: 0.3435
Epoch 2/2
992/992 [==============================] - 836s 843ms/step - loss: nan - accuracy: 0.3435
3 of 81 85.01108439366023 36.36585286319256
Epoch 1/2
980/980 [==============================] - 834s 852ms/step - loss: nan - accuracy: 0.3428
Epoch 2/2
980/980 [==============================] - 836s 854ms/step - loss: nan - accuracy: 0.3428
4 of 81 112.91956749757131 35.75786317004098
Epoch 1/2
967/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



896/896 [==============================] - 769s 858ms/step - loss: nan - accuracy: 0.3417
Epoch 2/2
896/896 [==============================] - 764s 852ms/step - loss: nan - accuracy: 0.3417
11 of 81 296.44883483250936 30.992378205879767
Epoch 1/2
885/885 [==============================] - 745s 842ms/step - loss: nan - accuracy: 0.3419
Epoch 2/2
575/885 [==================>...........] - ETA: 4:21 - loss: nan - accuracy: 0.3425

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



862/862 [==============================] - 732s 849ms/step - loss: nan - accuracy: 0.3414
Epoch 2/2
862/862 [==============================] - 725s 841ms/step - loss: nan - accuracy: 0.3414
14 of 81 370.4668197512627 29.108107279132284
Epoch 1/2
852/852 [==============================] - 693s 814ms/step - loss: nan - accuracy: 0.3418
Epoch 2/2
852/852 [==============================] - 716s 840ms/step - loss: nan - accuracy: 0.3418
15 of 81 393.98777869939806 28.454672919004054
Epoch 1/2
196/841 [=====>........................] - ETA: 8:55 - loss: nan - accuracy: 0.3439

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



581/581 [==============================] - 495s 853ms/step - loss: nan - accuracy: 0.3416
45 of 81 992.1719285845757 12.861487965171724
Epoch 1/2
574/574 [==============================] - 487s 848ms/step - loss: nan - accuracy: 0.3451
Epoch 2/2
574/574 [==============================] - 488s 850ms/step - loss: nan - accuracy: 0.3451
46 of 81 1008.4486707528432 12.422918409664849
Epoch 1/2
567/567 [==============================] - 480s 847ms/step - loss: nan - accuracy: 0.3405
Epoch 2/2
567/567 [==============================] - 485s 855ms/step - loss: nan - accuracy: 0.3405
47 of 81 1024.5594787597656 11.989525816868946
Epoch 1/2
140/559 [======>.......................] - ETA: 6:05 - loss: nan - accuracy: 0.3468

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



552/552 [==============================] - 457s 827ms/step - loss: nan - accuracy: 0.3395
49 of 81 1056.0484242796897 11.135204475730186
Epoch 1/2
545/545 [==============================] - 464s 851ms/step - loss: nan - accuracy: 0.3417
Epoch 2/2
545/545 [==============================] - 469s 861ms/step - loss: nan - accuracy: 0.3417
50 of 81 1071.6258803248406 10.716258804533217
Epoch 1/2
  6/538 [..............................] - ETA: 8:19 - loss: nan - accuracy: 0.3496

KeyboardInterrupt: 

In [ ]:
np.shape(Train_data[1])

In [ ]:
scores = lstm_autoencoder.predict(Test_data)

In [ ]:
lstm_autoencoder.plot(scores, Test_data, threshold=0.95)

In [ ]:

lstm_autoencoder.model.save("LSTM_Big")